In [1]:
import cv2
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.ndimage import label
import imageio
from IPython.display import Image
from scipy import interpolate

path = 'Fantastic Driver Slow Motion Swings of World Top 10.mp4'
cap = cv2.VideoCapture(path)

In [2]:
Count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
Width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
Height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [3]:
d = {0:{'start_frame':200,'end_frame':600,'x1':50,'y1':0,'x2':720,'y2':620},
     1:{'start_frame':700,'end_frame':1000,'x1':50,'y1':670,'x2':720,'y2':1280},
     2:{'start_frame':1100,'end_frame':1500,'x1':50,'y1':0,'x2':720,'y2':620},
     3:{'start_frame':1600,'end_frame':1900,'x1':50,'y1':670,'x2':720,'y2':1280},
     4:{'start_frame':2000,'end_frame':2400,'x1':50,'y1':0,'x2':720,'y2':620}}

In [4]:
# def nothing(x):
#     pass

# # Create a window
# cv2.namedWindow('image')

# # create trackbars for color change
# cv2.createTrackbar('HMin','image',0,255,nothing)
# cv2.createTrackbar('LMin','image',0,255,nothing)
# cv2.createTrackbar('SMin','image',0,255,nothing)
# cv2.createTrackbar('HMax','image',0,255,nothing)
# cv2.createTrackbar('LMax','image',0,255,nothing)
# cv2.createTrackbar('SMax','image',0,255,nothing)

# # Set default value for MAX HSV trackbars.
# cv2.setTrackbarPos('HMax', 'image', 255)
# cv2.setTrackbarPos('LMax', 'image', 255)
# cv2.setTrackbarPos('SMax', 'image', 255)

# # Initialize to check if HSV min/max value changes
# hMin = sMin = lMin = hMax = sMax = lMax = 0
# phMin = psMin = plMin = phMax = psMax = plMax = 0

# img = res[0,:,:,:]
# output = img
# waitTime = 33

# while(1):
#     # get current positions of all trackbars
#     hMin = cv2.getTrackbarPos('HMin','image')
#     lMin = cv2.getTrackbarPos('LMin','image')
#     sMin = cv2.getTrackbarPos('SMin','image')
    
#     hMax = cv2.getTrackbarPos('HMax','image')
#     lMax = cv2.getTrackbarPos('LMax','image')
#     sMax = cv2.getTrackbarPos('SMax','image')
    
#     # Set minimum and max HSV values to display
#     lower = np.array([hMin, lMin, sMin])
#     upper = np.array([hMax, lMax, sMax])

#     # Create HSV Image and threshold into a range.
#     hls = cv2.cvtColor(img[:,:,[2,1,0]], cv2.COLOR_BGR2HLS)
    
#     mask = cv2.inRange(hls, lower, upper)
#     output = cv2.bitwise_and(img, img, mask=mask)

#     # Display output image
#     cv2.imshow('image',output)

#     # Wait longer to prevent freeze for videos.
#     if cv2.waitKey(waitTime) & 0xFF == ord('q'):
#         break

# # cv2.destroyAllWindows()

In [5]:
def filter_img(img):
    # lower_white = np.array([49,0,120], dtype=np.uint8)
    # upper_white = np.array([130, 80, 255], dtype=np.uint8)
    # hsv = cv2.cvtColor(img[:,:,[2,1,0]], cv2.COLOR_BGR2HSV)
    
    hls = cv2.cvtColor(img[:,:,[2,1,0]], cv2.COLOR_RGB2HLS)
    
    lower_white = np.array([0,200,0], dtype=np.uint8)
    upper_white = np.array([255, 255, 255], dtype=np.uint8)
    white = cv2.inRange(hls, lower_white, upper_white)
    h,w = white.shape
    white[0:int(0.75*h),:]=0
    white[:,0:int(0.75*w)]=0
    
    white[0:int(0.75*h),:]=0
    white[:,0:int(0.75*w)]=0

    return white, hls[int(0.75*h):,int(0.75*w):]

def find_ball(img):
    
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    lab_planes = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    img = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

    white, hls = filter_img(img)
    
    bgr_planes = cv2.split(hls)
    h = cv2.calcHist(bgr_planes, [0], None, [256], [0, 256], accumulate=False)
    l = cv2.calcHist(bgr_planes, [1], None, [256], [0, 256], accumulate=False)
    s = cv2.calcHist(bgr_planes, [2], None, [256], [0, 256], accumulate=False)
    
    labeled_array, num_features = label(white)
    islands, cnts = np.unique(labeled_array, return_counts = True)
    ball_like_img = labeled_array.copy()
    
    for idx, cnt in zip(islands, cnts):
        if (cnt > 150) or (cnt < 50):
            ball_like_img[ball_like_img==idx] = 0
    ball_like_img[ball_like_img!=0] = 255
    
    contours, hierarchy = cv2.findContours(ball_like_img.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    center = (-1, -1)
    
    for idx, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, closed=True)
        borders = cv2.approxPolyDP(curve=contour,epsilon=0.01*perimeter,closed=True)
        
        if len(borders) > 4: # Circle
            (x1,y1),r = cv2.minEnclosingCircle(contour)
            area_circle = r*r*np.pi
            if area / area_circle > 0.5:
                center = (x1,y1)
                img = cv2.drawContours(img, [contour], -1, color=(255, 0, 0), thickness=cv2.FILLED)
                img = cv2.circle(img, (int(x1), int(y1)), int(r), (0,255,0), 2)   
    
    return img, white, ball_like_img, center, (h,l,s), hls

def change_brightness(img, beta):
    # img = cv2.convertScaleAbs(img, img, 1, beta)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v = img[:, :, 2]
    v = np.where(v <= 255 - beta, v + beta, 255)
    img[:, :, 2] = v
    img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)

    return img

def get_video(id_,path,d, b=False):
    start_frame, end_frame = d[id_]['start_frame'], d[id_]['end_frame']
    x1, x2, y1, y2 = d[id_]['x1'], d[id_]['x2'], d[id_]['y1'], d[id_]['y2']

    cap = cv2.VideoCapture(path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame - 1)
    res = np.empty((end_frame - start_frame, Height, Width, 3)).astype(np.uint8)
    idx = 0

    while(idx < end_frame - start_frame):
        ret, frame = cap.read()
        frame = frame[:,:,[2,1,0]]
        if not ret:
            print('No frames grabbed!')
            break
        if b:
            frame = change_brightness(frame, -20)
        res[idx,:,:,:] = frame
        idx +=1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    res = res[:,x1:x2,y1:y2,:]

    cap.release()
    return res

In [6]:
id_=0
res = get_video(id_,path,d)
img, white, ball_like_img, center, (h,l,s), hls = find_ball(res[0,:,:,:].copy())

plt.figure(figsize=(20,7))
plt.subplot(1,4,1)
plt.imshow(white)
plt.subplot(1,4,2)
plt.imshow(img)
plt.subplot(1,4,3)
plt.imshow(hls)
plt.subplot(1,4,4)
# plt.plot(h,'red', label='Hue')
plt.plot(l,'blue', label='Lightness')
# plt.plot(s,'green', label='Saturation')
plt.legend()
plt.xlim([0, 256])

plt.show()

TypeError: 'tuple' object does not support item assignment

In [ ]:
tracking_orig = res.copy()
center_pts = []
for idx_img, img in enumerate(tqdm(tracking_orig)):
    result_img, white, ball_like_img, center, (h,l,s), hls = find_ball(img)
    center_pts.append(center)
    tracking_orig[idx_img,:,:,:] = result_img

pts_range = np.arange(len(center_pts))
x = np.array(center_pts)[:,0]
y = np.array(center_pts)[:,1]

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.plot(pts_range, x, "x")
plt.subplot(1,2,2)
plt.plot(pts_range, y, "x")
plt.show()

In [ ]:
id_=1
res = get_video(id_,path,d)
img, white, ball_like_img, center, (h,l,s), hls = find_ball(res[0,:,:,:].copy())
print(sum(l[200:]))

plt.figure(figsize=(20,7))
plt.subplot(1,4,1)
plt.imshow(white)
plt.subplot(1,4,2)
plt.imshow(img)
plt.subplot(1,4,3)
plt.imshow(hls)
plt.subplot(1,4,4)
# plt.plot(h,'red', label='Hue')
plt.plot(l,'blue', label='Lightness')
# plt.plot(s,'green', label='Saturation')
plt.legend()
plt.xlim([0, 256])
plt.show()

In [ ]:
tracking_orig = res.copy()
center_pts = []
for idx_img, img in enumerate(tqdm(tracking_orig)):
    result_img, white, ball_like_img, center, (h,l,s), hls = find_ball(img)
    center_pts.append(center)
    tracking_orig[idx_img,:,:,:] = result_img

pts_range = np.arange(len(center_pts))
x = np.array(center_pts)[:,0]
y = np.array(center_pts)[:,1]

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.plot(pts_range, x, "x")
plt.subplot(1,2,2)
plt.plot(pts_range, y, "x")
plt.show()

In [ ]:
id_=2
res = get_video(id_,path,d)
img, white, ball_like_img, center, (h,l,s), hls = find_ball(res[0,:,:,:].copy())
print(sum(l[200:]))

plt.figure(figsize=(20,7))
plt.subplot(1,4,1)
plt.imshow(white)
plt.subplot(1,4,2)
plt.imshow(img)
plt.subplot(1,4,3)
plt.imshow(hls)
plt.subplot(1,4,4)
# plt.plot(h,'red', label='Hue')
plt.plot(l,'blue', label='Lightness')
# plt.plot(s,'green', label='Saturation')
plt.legend()
plt.xlim([0, 256])
plt.show()

In [ ]:
tracking_orig = res.copy()
center_pts = []
for idx_img, img in enumerate(tqdm(tracking_orig)):
    result_img, white, ball_like_img, center, (h,l,s), hls = find_ball(img)
    center_pts.append(center)
    tracking_orig[idx_img,:,:,:] = result_img

pts_range = np.arange(len(center_pts))
x = np.array(center_pts)[:,0]
y = np.array(center_pts)[:,1]

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.plot(pts_range, x, "x")
plt.subplot(1,2,2)
plt.plot(pts_range, y, "x")
plt.show()

In [ ]:
id_=3
res = get_video(id_,path,d)
img, white, ball_like_img, center, (h,l,s), hls = find_ball(res[0,:,:,:].copy())
print(sum(l[200:]))

plt.figure(figsize=(20,7))
plt.subplot(1,4,1)
plt.imshow(white)
plt.subplot(1,4,2)
plt.imshow(img)
plt.subplot(1,4,3)
plt.imshow(hls)
plt.subplot(1,4,4)
# plt.plot(h,'red', label='Hue')
plt.plot(l,'blue', label='Lightness')
# plt.plot(s,'green', label='Saturation')
plt.legend()
plt.xlim([0, 256])
plt.show()

In [ ]:
tracking_orig = res.copy()
center_pts = []
for idx_img, img in enumerate(tqdm(tracking_orig)):
    result_img, white, ball_like_img, center, (h,l,s), hls = find_ball(img)
    center_pts.append(center)
    tracking_orig[idx_img,:,:,:] = result_img

pts_range = np.arange(len(center_pts))
x = np.array(center_pts)[:,0]
y = np.array(center_pts)[:,1]

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.plot(pts_range, x, "x")
plt.subplot(1,2,2)
plt.plot(pts_range, y, "x")
plt.show()

In [ ]:
id_=4
res = get_video(id_,path,d)
img, white, ball_like_img, center, (h,l,s), hls = find_ball(res[0,:,:,:].copy())
print(sum(l[200:]))

plt.figure(figsize=(20,7))
plt.subplot(1,4,1)
plt.imshow(white)
plt.subplot(1,4,2)
plt.imshow(img)
plt.subplot(1,4,3)
plt.imshow(hls)
plt.subplot(1,4,4)
# plt.plot(h,'red', label='Hue')
plt.plot(l,'blue', label='Lightness')
# plt.plot(s,'green', label='Saturation')
plt.legend()
plt.xlim([0, 256])
plt.show()

In [ ]:
tracking_orig = res.copy()
center_pts = []
for idx_img, img in enumerate(tqdm(tracking_orig)):
    result_img, white, ball_like_img, center, (h,l,s), hls = find_ball(img)
    center_pts.append(center)
    tracking_orig[idx_img,:,:,:] = result_img

pts_range = np.arange(len(center_pts))
x = np.array(center_pts)[:,0]
y = np.array(center_pts)[:,1]

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.plot(pts_range, x, "x")
plt.subplot(1,2,2)
plt.plot(pts_range, y, "x")
plt.show()